# Caso final de repaso PLN: Solución

## Contexto

La agencia de comunicación del Ministerio de Turismo de España, nos ha contactado porque quieren monitorizar que se dice de nuestro país en redes sociales y ver la opinión general y el tono de los mensajes. 
Para ello nos han pedido realizar un script donde se pueda evaluar la positividad o negatividad de los mensajes en redes sociales.
<br>
Como consultores les hemos propuesto una solución que analizará cada tweet publicado y le dará una puntuación entre -100% y 100% donde los valores superiores a 0% nos indican que es positivo y los valores inferiores a 0% negativos, de igual manera el 0% será neutro.
<br>
Para realizar una primera prueba, la agencia nos ha proporcionado lo siguiente:
-	Archivo CSV con 500 tweets
-	Archivo CSV con palabras negativas
-	Archivo CSV con palabras positivas
<br><br>
Con esta información y materiales ya podemos realizar la tarea encomendada.<br><br>

**Nota:** Este ejemplo se ha realizado utilizando los archivos extendidos de palabras


## Solución al caso práctico

### Importación de librerías

In [1]:
import pandas as pd
pd.set_option('max_colwidth', -1)
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

import spacy
nlp = spacy.load('es_core_news_sm')

C:\Users\Juan Carlos\AppData\Local\Temp\ipykernel_7832\1129062342.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


### Funciones para detectar la polaridad de las palabras
<br>
En primer lugar debemos crear 2 funciones que detecten cuántas palabras positivas y cuántas palabras negativas contiene una frase. 
<br><br>
Estas funciones recibirán los tokens que componen las frases a analizar, en nuestro caso cada texto del tweet.
<br><br>

#### Detectar las palabras positivas

In [2]:
def positivas(tokens):
    
    #Primero abrimos el fichero y eliminamos carácteres especiales
    with open('positivas_ext.csv', encoding="ANSI") as f:
        content = f.read() 
        #content = re.sub('[,\.()":;!@#$%^&*\d]|\'s|\'', '', content)
        p_list = content.replace('\n',' ').replace('  ',' ').lower().split(' ')
        
    cleaned_words = [] #Este array nos servirá para tener los lemas de las palabras positivas    
    separator = ' '
        
    for token in nlp(separator.join(p_list)):
        cleaned_words.append(token.lemma_)
    
    #Inicializamos un contador de palabras a 0 para sumar 1 cada vez que se encuentre una en la lista
    conteo = 0
    
    #Si el token está en la lista sumamos 1
    for token in tokens:
        if token in cleaned_words:
            conteo += 1
            print('POS: ' + token) #Este print puedes comentarlo para ocultar las palabras
    
    return conteo #Devolvemos la cantidad de palabras encontradas

#### Detectar palabras negativas

Ahora simplemente tenemos que duplicar la función que tenemos para las positivas y cambiar el CSV que utilizamos para comparar

In [3]:
def negativas(tokens):
    
    #Primero abrimos el fichero y eliminamos carácteres especiales
    with open('negativas_ext.csv', encoding="ANSI") as f:
        content = f.read() 
        #content = re.sub('[,\.()":;!@#$%^&*\d]|\'s|\'', '', content)
        p_list = content.replace('\n',' ').replace('  ',' ').lower().split(' ')
        
    cleaned_words = [] #Este array nos servirá para tener los lemas de las palabras negativas    
    separator = ' '
        
    for token in nlp(separator.join(p_list)):
        cleaned_words.append(token.lemma_)
    
    #Inicializamos un contador de palabras a 0 para sumar 1 cada vez que se encuentre una en la lista
    conteo = 0
    
    #Si el token está en la lista sumamos 1
    for token in tokens:
        if token in cleaned_words:
            conteo += 1
            print('NEG: ' + token) #Este print puedes comentarlo para ocultar las palabras
    
    return conteo #Devolvemos la cantidad de palabras encontradas

### Tokenizar frases

Ahora tenemos que crear una función para tokenizar las frases que tenemos, eliminar las stopwords, y estandarizar las palabras. De esta forma nos quedaremos únicamente con las palabras que nos interesan y le dan significado a la frase.

#### Tokenizar y limpiar frases

In [4]:
def tokenizar(frase):
    
    #Tokenizar
    tokens = word_tokenize(frase, "spanish")
    
    #Eliminar carácteres especiales, números, emojis, links ...
    tokens_alpha = [word.lower() for word in tokens if word.isalpha()]
    
    #Eliminar StopWords
    tokens_stopW = tokens_alpha[:]
    for token in tokens_alpha:
        if token in stopwords.words('spanish'):
            tokens_stopW.remove(token)
            
    #Obtener Lemas    
    cleaned_tokens = []    
    separator = ' '
        
    for token in nlp(separator.join(tokens_stopW)):
        cleaned_tokens.append(token.lemma_)
        
    
    return cleaned_tokens

### Función para detectar la polaridad

Ahora ya si toca crear la función para derectar la polaridad. Esta función agrupará las anteriores.

In [5]:
def polaridad(frase):
    
    # 1.- Tokenizamos y limpiamos la frase
    tokens = tokenizar(frase)
    
    # 2.- Obtenemos el número de tokens válidos
    tokens_num = len(tokens)
    
    # 3.- Obtenemos el número de palabras positivas y negativas de la frase
    positive_num = positivas(tokens)
    negative_num = negativas(tokens)
    
    # 4.- Calculamos la puntuación de positividad y negatividad
    postVal = positive_num / tokens_num
    negVal = (negative_num / tokens_num)*-1
    
    # 5.- Calculamos el valor final de nuesta función
    polarity = negVal + postVal
    
    # 6.- Damos el formato final de porcentaje al resultado
    polarityForm = "{:.2%}".format(polarity)
    
    return polarityForm

### Probamos el funcionamiento en primer lugar con algunas frases de ejemplo

In [6]:
frase1 = 'El castillo ha sido conquistado violentamente en la batalla de trafalgar'
frase2 = 'Es sorprendente lo amistoso e inteligente que es este gato. Es maravilloso'

print('Ejemplo 1')
print('Frase: ' + frase1)
print('Polaridad: ' + str(polaridad(frase1)))
print('-------------------------')
print('Ejemplo 2')
print('Frase: ' + frase2)
print('Polaridad: ' + str(polaridad(frase2)))

Ejemplo 1
Frase: El castillo ha sido conquistado violentamente en la batalla de trafalgar
NEG: ser
NEG: violentamente
NEG: batalla
Polaridad: -50.00%
-------------------------
Ejemplo 2
Frase: Es sorprendente lo amistoso e inteligente que es este gato. Es maravilloso
POS: sorprendente
POS: amistoso
POS: inteligente
POS: maravilloso
Polaridad: 80.00%


### Prueba con los tweets

Ahora cargamos el dataset con los tweets y analizamos cada uno de ellos

In [7]:
tweets = pd.read_csv('tweets_dataset.csv')
tweets.iloc[:5] #Mostramos los 5 primeros valores

,index,ID,text
0,0,1459083556286713863,¡Vamos a sortear una AMD Radeon RX 6900 XT Halo Infinite Limited Edition Graphics Card! 🐍\nRequisitos para parcicipa… https://t.co/F3TzO6dhKr
1,1,1458823365674864644,"En un país normal, proponer para el Constitucional a un tipo envuelto en casos de corrupción, que niega el cambio c… https://t.co/WjU7cUhb2D"
2,2,1459060580615176193,"🔹 11 suicidios al día\n\n🔹 Una persona se quita la vida cada 2 horas\n\n🔹 Entre ellos, el doble de niños que en 2019\n\nE… https://t.co/GpM02UQ9dv"
3,3,1459098007492870147,Sánchez continúa su farsa de recuperación mientras las previsiones económicas sobre España se desploman y la cesta… https://t.co/VTygFpxduV
4,4,1458881721919750149,"Sr. Sánchez, si todo es tan maravilloso, si usted es tan querido por los españoles, si España está tan bien, y los… https://t.co/sDTrOBjhAy"


Creamos un bucle para recorrer todos los tweets del archivo y analizar su polaridad. <br><br>
Con esto ya tendríamos completado lo solicitado

In [8]:
row = 1
while row < len(tweets)-1:
    
    tweet_text = tweets.iloc[row]['text']
    
    print('Tweet ' + str(tweets.iloc[row]['ID']))
    print('Contenido del tweet: ')
    print(tweet_text)
    print(' ')   
    
    polarity = polaridad(tweet_text)
    
    
    print(' ')
    print('La polaridad del texto es: ' + str(polarity))
    print('-------------------------------------')
    
    row += 1

Tweet 1458823365674864644
Contenido del tweet: 
En un país normal, proponer para el Constitucional a un tipo envuelto en casos de corrupción, que niega el cambio c… https://t.co/WjU7cUhb2D
 
NEG: tipo
NEG: envuelto
NEG: negar
 
La polaridad del texto es: -27.27%
-------------------------------------
Tweet 1459060580615176193
Contenido del tweet: 
🔹 11 suicidios al día

🔹 Una persona se quita la vida cada 2 horas

🔹 Entre ellos, el doble de niños que en 2019

E… https://t.co/GpM02UQ9dv
 
NEG: suicidio
NEG: persona
NEG: vida
 
La polaridad del texto es: -30.00%
-------------------------------------
Tweet 1459098007492870147
Contenido del tweet: 
Sánchez continúa su farsa de recuperación mientras las previsiones económicas sobre España se desploman y la cesta… https://t.co/VTygFpxduV
 
 
La polaridad del texto es: 0.00%
-------------------------------------
Tweet 1458881721919750149
Contenido del tweet: 
Sr. Sánchez, si todo es tan maravilloso, si usted es tan querido por los españoles, s

 
La polaridad del texto es: 0.00%
-------------------------------------
Tweet 1458408054261784582
Contenido del tweet: 
⚠️ ATENCIÓN⚠️ 50 euros para gastar en amazon (España) van a ser para una de las personas que haga RT 🔁 a este tweet… https://t.co/at27OAW76P
 
NEG: ser
NEG: persona
NEG: hacer
 
La polaridad del texto es: -30.00%
-------------------------------------
Tweet 1458480406307741696
Contenido del tweet: 
Sois seres humanos maravillosos todos los que estáis poniendo y compartiendo el hastag #SOSenfermosELA 

Hay que ll… https://t.co/QXllRRQC36
 
POS: humano
POS: maravilloso
NEG: ser
NEG: poner
 
La polaridad del texto es: 0.00%
-------------------------------------
Tweet 1458492836509274116
Contenido del tweet: 
Gobierno de España, partidos de la oposición, partidos políticos en general.

Los enfermos de ELA estamos obligados… https://t.co/4tVWWPdnM1
 
NEG: general
NEG: enfermo
 
La polaridad del texto es: -20.00%
-------------------------------------
Tweet 14580034024960368

POS: llamar
POS: bendecir
 
La polaridad del texto es: 22.22%
-------------------------------------
Tweet 1456689425769979906
Contenido del tweet: 
Por partes:
A. Eso no es un plató, es una sala de espera.
B. @vox_es, la 3ra fuerza política de España,va a ir a to… https://t.co/GuV6caop9L
 
NEG: parte
 
La polaridad del texto es: -9.09%
-------------------------------------
Tweet 1456143988231188482
Contenido del tweet: 
🔴 El Gobierno de Pedro Sánchez "no repara en gastos" y bate el gasto histórico en asesores: 48,3 millones en nueve… https://t.co/BsvW4OdsYH
 
 
La polaridad del texto es: 0.00%
-------------------------------------
Tweet 1456136502358970369
Contenido del tweet: 
Irene Montero gasta 70.000 € en las fotos y vídeos de sus actos como ministra https://t.co/qvuX3SLClW
 
 
La polaridad del texto es: 0.00%
-------------------------------------
Tweet 1456137625539158020
Contenido del tweet: 
Sánchez evita explicar por qué hizo un Barcelona-Zaragoza en Falcon y no en AVE: «Es sec

## Conclusión

Como has observado no es demasiado complejo realizar este analizador de la polaridad, la verdadera complejidad reside en mejorar el listado de palabras. <br><br>
Puedes probar a eliminar o añadir palabras al listado para mejorar su resultado. También puedes crear tu propio listado desde 0.
<br><br>
Te animamos a mejorar el listado y compartirlo con tu tutor para dejarlo disponible en los recursos para que otros compañeros puedan probar con distintos dataset de palabras.